In [2]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re

In [1]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup

# Web Scraping Data From Wikipedia
Most cities in the data set have very informative Wikipedia Sites. This data is publicaly avialable to anyone with a internet connection. As we're are particular interested in the comparing the CO2 emision with climate/geological data we're partilcuar interested in the section on Wikipedia page which describe a city's cliamte.  Below can a screenshot be seen of the Climate Data table for Berlin. 
![title](Berlin_climate.png)
Similar tables exisist for almost all the cities in the dataset. This data can be gatehered using the `BeautifulSoup` library. 

In [3]:
df = pd.read_csv(os.path.abspath('../data/Processed/Cities_imputed.csv'), index_col=0)

In [4]:
climate_df = df[['City', 'Country', 'Continent']]

In [5]:
climate_df['wiki_link'] = [re.sub(r'\([^)]*\)', '', city).replace(" ", "_") for city in climate_df.City]

In [6]:
climate_df.loc[climate_df['wiki_link'] == 'Birmingham']

,City,Country,Continent,wiki_link
319,Birmingham(AL),United States,North America,Birmingham
261,Birmingham,United Kingdom,Europe,Birmingham


## Correcting for the name of the city does land on the right Wikipage

In [7]:
def update_wiki_link(df, city, city_update):
    if city in df.wiki_link.values:
        df.loc[df['wiki_link'] == city, 'wiki_link'] = city_update
    else:
        print(city)

In [8]:
cities_updates = [
    ('Santa_Cruz', 'Santa_Cruz_(Bolivia)'),
    ('Cordoba' , 'Córdoba,_Argentina'),
    ('San_Francisco_Bay_Area', 'San_Fransisco'),
    ('Tampa-St._Petersburg', 'Saint_Petersburg_(Florida)'),
    ('Memphis', 'Memphis,_Tennessee'),
    ('Denver-Aurora', 'Aurora,_Colorado'),
    ('Rotterdam-Hague', 'Rotterdam'),
    ('Nashville-Davidson', 'Nashville,_Tennessee'),
    ('Cologne-Bonn', 'Cologne'),
    ('Richmond', 'Richmond,_Virginia'),
    ('Rochester', 'Rochester,_New_York'),
    ('Phoenix-Mesa', 'Phoenix_(Arizona)'),
    ('Leon', 'León,_Guanajuato'),
    ('Buffalo', 'Buffalo,_New_York'),
    ('Allentown-Bethlehem', 'Allentown,_Pennsylvania'),
    ('San_Juan', 'San_Juan,_Puerto_Rico'),
    ('Puebla', 'Puebla_(city)'),
    ('San_Jose', 'San_José,_Costa_Rica'),
    ('Dallas-Fort_Worth', 'Fort_Worth,_Texas'),
    ('Seoul-Incheon', 'Seoul'),
    ('Lome', 'Lomé'),
    ('Chihuahua', 'Chihuahua_City'),
    ('Fukuoka-Kitakyushu', 'Fukuoka'),
    ('Davao', 'Davao_City'),
    ('New_York', 'New_York_City'),
    ('Osaka-Kobe-Kyoto', 'Osaka'),
    ('Kansas_City', 'Kansas_City,_Missouri'),
    ('Minneapolis-St._Paul', 'Minneapolis'),
    ('Portland', 'Portland,_Oregon'),
    ('Phoenix_(Arizona)', 'Phoenix,_Arizona'),
    ('Washington', 'Washington,_D.C.'),
    ('Columbus', 'Columbus,_Ohio'),
    ('Salvador', 'Salvador,_Bahia'),
    ('Xi‚Äôan', 'Xi%27an'),
    ('Concepcion', 'Concepción,_Chile'),
    ('Providence', 'Providence,_Rhode_Island')
]

for city, city_update in cities_updates:
    update_wiki_link(
        df = climate_df,
        city = city,
        city_update = city_update
    )

In [9]:
climate_df.loc[(climate_df['wiki_link'] == 'Birmingham') & (climate_df['Country'] == 'United States')] = 'Birmingham,_Alabama'
climate_df.loc[(climate_df['wiki_link'] == 'Valencia') & (climate_df['Country'] == 'United Venezuela')] = 'Valencia,_Carabobo'

In [10]:
def extract_climate_table(soup):
    # exstract the table with the cliamte data
    tables = soup.findAll('table',{"class":"collapsible"})

    select_table = None
    for table in tables:
        header = table.findAll(['th'])[0].getText()[0:12].lower()
        
        if 'climate' in header:
            select_table = table
            break
    
    return select_table

In [11]:
climate_data_dict = dict()

for wiki_link in climate_df.wiki_link:
    url = 'https://en.wikipedia.org/wiki/' + wiki_link

    response = requests.get(url)
    status_code = response.status_code
    if status_code != 200:
        print('Could not find city: ', wiki_link, 'Wikipedia')
        continue

    soup = BeautifulSoup(response.text,"html.parser")

    # exstract the table with the cliamte data
    table = extract_climate_table(soup)

    # In case the city does not have a climate table
    if not table:
        climate_data_dict[wiki_link] = None
        print('No climate data available for city: ', wiki_link)
        continue

    trs = table.findAll(['tr'])[1:]

    for i, cell in enumerate(trs):
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]  
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]
        
        
        if i == 0:
            columns = th_data
            city_climate_df_tmp = pd.DataFrame(columns=columns)
        else:
            
            if len(th_data) == 0:
                continue
                
            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)

    city_climate_df_tmp.rename(columns={"Month": "Climate_variable"}, inplace = True)
    city_climate_df_tmp.set_index('Climate_variable', inplace = True)

    climate_data_dict[wiki_link] = city_climate_df_tmp


No climate data available for city:  Cebu
No climate data available for city:  Caracas
No climate data available for city:  Santo_Domingo
No climate data available for city:  Monterrey


## Processing the climate data

In [12]:
climate_data_dict['Austin']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °F (°C),90(32),99(37),98(37),99(37),104(40),109(43),109(43),112(44),112(44),100(38),91(33),90(32),112(44)
Mean maximum °F (°C),79.3(26.3),83.3(28.5),86.6(30.3),91.7(33.2),95.4(35.2),98.1(36.7),101.0(38.3),102.9(39.4),99.1(37.3),93.3(34.1),84.8(29.3),79.7(26.5),103.9(39.9)
Average high °F (°C),61.5(16.4),65.2(18.4),72.2(22.3),79.8(26.6),86.5(30.3),92.1(33.4),95.6(35.3),97.0(36.1),90.5(32.5),81.8(27.7),71.4(21.9),62.7(17.1),79.8(26.6)
Average low °F (°C),41.5(5.3),44.8(7.1),51.3(10.7),58.6(14.8),66.7(19.3),72.3(22.4),74.4(23.6),74.6(23.7),69.4(20.8),60.6(15.9),50.6(10.3),42.3(5.7),59.0(15.0)
Mean minimum °F (°C),26.7(−2.9),28.5(−1.9),34.7(1.5),42.9(6.1),54.1(12.3),64.2(17.9),69.9(21.1),69.0(20.6),56.9(13.8),44.7(7.1),34.6(1.4),26.7(−2.9),22.0(−5.6)
Record low °F (°C),−2(−19),−1(−18),18(−8),30(−1),40(4),51(11),57(14),58(14),41(5),30(−1),20(−7),4(−16),−2(−19)
Average precipitation inches (mm),2.22(56),2.02(51),2.76(70),2.09(53),4.44(113),4.33(110),1.88(48),2.35(60),2.99(76),3.88(99),2.96(75),2.40(61),34.32(872)
Average snowfall inches (cm),0.4(1.0),0.2(0.51),0(0),0(0),0(0),0(0),0(0),0(0),0(0),0(0),0(0),trace,0.6(1.5)
Average precipitation days (≥ 0.01 in),7.4,7.4,9.2,7.1,8.9,7.7,5.4,4.9,6.7,7.5,7.5,7.8,87.5


In [13]:
climate_data_dict['Freetown']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °C (°F),36.0(96.8),38.0(100.4),38.0(100.4),38.5(101.3),35.0(95.0),35.0(95.0),33.0(91.4),32.0(89.6),31.0(87.8),35.0(95.0),36.5(97.7),33.5(92.3),38.5(101.3)
Average high °C (°F),29.9(85.8),30.3(86.5),30.9(87.6),31.2(88.2),30.9(87.6),30.1(86.2),28.7(83.7),28.4(83.1),29.0(84.2),29.9(85.8),30.1(86.2),29.7(85.5),29.9(85.9)
Daily mean °C (°F),27.3(81.1),27.6(81.7),28.1(82.6),28.5(83.3),28.3(82.9),27.4(81.3),26.3(79.3),25.9(78.6),26.4(79.5),27.1(80.8),27.7(81.9),27.5(81.5),27.3(81.2)
Average low °C (°F),23.8(74.8),24.0(75.2),24.4(75.9),24.8(76.6),24.4(75.9),23.6(74.5),23.1(73.6),23.0(73.4),23.1(73.6),23.4(74.1),24.0(75.2),24.1(75.4),23.8(74.9)
Record low °C (°F),15.0(59.0),17.8(64.0),19.4(66.9),20.0(68.0),20.6(69.1),20.0(68.0),19.4(66.9),19.4(66.9),20.0(68.0),19.4(66.9),20.0(68.0),15.6(60.1),15.0(59.0)
Average rainfall mm (inches),8.0(0.31),6.0(0.24),28.0(1.10),68.0(2.68),214.0(8.43),522.0(20.55),"1,190(46.85)","1,078(42.44)",800.0(31.50),333.0(13.11),148.0(5.83),38.0(1.50),"4,433(174.54)"
Average rainy days,0,0,1,4,15,22,27,27,24,21,9,2,152
Average relative humidity (%) (at 15:00 LST),66,67,67,68,73,76,81,82,80,78,76,69,74
Mean monthly sunshine hours,226.3,215.6,232.5,207.0,189.1,153.0,102.3,86.8,126.0,186.0,198.0,161.2,"2,083.8"


In [14]:
city_climate_list = [list(climate_df.index.values) for city, climate_df in climate_data_dict.items() if climate_df is not None]
climate_variables = [city for sublist in city_climate_list for city in sublist]
set_of_climate_variable = set(climate_variables)

In [15]:
set_of_climate_variable

{'Average afternoon relative humidity (%)',
 'Average dew point °C (°F)',
 'Average dew point °F (°C)',
 'Average high °C (°F)',
 'Average high °F (°C)',
 'Average low °C (°F)',
 'Average low °F (°C)',
 'Average precipitation days',
 'Average precipitation days (≥ 0.005 inch)',
 'Average precipitation days (≥ 0.01 in)',
 'Average precipitation days (≥ 0.01 in.)',
 'Average precipitation days (≥ 0.01 inch)',
 'Average precipitation days (≥ 0.01\xa0mm)',
 'Average precipitation days (≥ 0.1 mm)',
 'Average precipitation days (≥ 0.1\xa0mm)',
 'Average precipitation days (≥ 0.2 mm)',
 'Average precipitation days (≥ 0.2\xa0mm)',
 'Average precipitation days (≥ 0.5 mm)',
 'Average precipitation days (≥ 1.0 mm)',
 'Average precipitation days (≥ 1\xa0mm)',
 'Average precipitation inches (mm)',
 'Average precipitation mm (inches)',
 'Average rainfall inches (mm)',
 'Average rainfall mm (inches)',
 'Average rainy days',
 'Average rainy days (≥ 0.01 in)',
 'Average rainy days (≥ 0.01 inch)',
 'Ave

In [36]:
rename_column = {
    'Average high °C (°F)'  : 'Average high °C (°F)',
    'Average high °F (°C)'  : 'Average high °C (°F)',
    'Average low °C (°F)'   : 'Average low °C (°F)',
    'Average low °F (°C)'   : 'Average low °C (°F)',
    'Average rainfall inches (mm)'      : 'Average precipitation mm',
    'Average rainfall mm (inches)'      : 'Average precipitation mm',
    'Average precipitation inches (mm)' : 'Average precipitation mm',
    'Average precipitation mm (inches)' : 'Average precipitation mm',
    'Average precipitation days'                : 'Average precipitation days',
    'Average precipitation days (≥ 0.005 inch)' : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 in)'    : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 in.)'   : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 inch)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.01\xa0mm)' : 'Average precipitation days',
    'Average precipitation days (≥ 0.1 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 0.1\xa0mm)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.2 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 0.2\xa0mm)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.5 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 1.0 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 1\xa0mm)'    : 'Average precipitation days',
    'Average rainy days'                        : 'Average precipitation days',
    'Average rainy days (≥ 0.01 in)'            : 'Average precipitation days',
    'Average rainy days (≥ 0.01 inch)'          : 'Average precipitation days',
    'Average rainy days (≥ 0.01 mm)'            : 'Average precipitation days',
    'Average rainy days (≥ 0.1 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 0.2 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 0.5 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 1 mm)'               : 'Average precipitation days',
    'Average rainy days (≥ 1.0 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 1.0\xa0mm)'          : 'Average precipitation days',
    'Average rainy days (≥ 1mm)'                : 'Average precipitation days',
}

In [37]:
def select_celcius(row):

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            matric.append(min(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric

In [38]:
def select_mm(row):

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            matric.append(max(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric

In [39]:
function_dict = {
    'Average high °C (°F)' : select_celcius,
    'Average low °C (°F)' : select_celcius,
    'Average rainfall mm (inches)' : select_mm,
    'Average rainy days' : lambda x : x,
}

In [50]:
cleaned_city_dict = dict()
for i, (city, climate_df) in enumerate(climate_data_dict.items()):

    if climate_df is None:
        continue

    # Rename remaing climate variables:
    city_climate_variables = climate_df.index.tolist()
    name_change  = {climate_variable : rename_column[climate_variable] for climate_variable  in city_climate_variables if climate_variable in rename_column}
    climate_df.rename(columns = name_change, inplace = True)

    # only keep the variables where the name was updated:
    climate_df_update = climate_df.loc[name_change.keys()]

    # Ensure only metrix represenation is kept:
    columns = climate_df_update.columns
    metric_climate_df = pd.DataFrame(columns = columns)

    climate_variables = []
    for climate_var, row in climate_df_update.iterrows():
        climate_variables.append(climate_var)
        metric = function_dict[climate_var](climate_df_update.loc[climate_var])

        tmp = pd.DataFrame({key: [value] for key, value in zip(columns, metric)})
        metric_climate_df = metric_climate_df.append(tmp)

    metric_climate_df['Climate_variable'] = climate_variables
    metric_climate_df.set_index('Climate_variable', inplace = True)

    # Update the city dataframe such that it only keeps the choosen climate variables
    # and they are metric
    cleaned_city_dict[city] = metric_climate_df

KeyError: 'Average high °F (°C)'

In [52]:
name_change

{'Average high °F (°C)': 'Average high °C (°F)',
 'Average low °F (°C)': 'Average low °C (°F)',
 'Average precipitation inches (mm)': 'Average precipitation mm',
 'Average precipitation days (≥ 0.01 in)': 'Average precipitation days'}

In [53]:
climate_df.rename(columns = name_change)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °F (°C),42.4(5.8),45.7(7.6),53.9(12.2),65.5(18.6),75.2(24.0),85.0(29.4),89.0(31.7),87.0(30.6),80.3(26.8),68.4(20.2),57.7(14.3),46.1(7.8),66.4(19.1)
Average low °F (°C),29.2(−1.6),31.4(−0.3),38.8(3.8),47.6(8.7),56.9(13.8),67.1(19.5),72.5(22.5),70.9(21.6),63.8(17.7),51.8(11.0),42.9(6.1),32.8(0.4),50.5(10.3)
Average precipitation inches (mm),2.92(74),2.60(66),3.86(98),3.22(82),3.49(89),3.27(83),4.62(117),3.39(86),4.09(104),3.05(77),2.97(75),3.41(87),"40.89(1,038)"
Average snowfall inches (cm),6.8(17),8.0(20),1.9(4.8),trace,0(0),0(0),0(0),0(0),0(0),0(0),0.4(1.0),3.0(7.6),20.1(51)
Average precipitation days (≥ 0.01 in),9.5,8.4,10.5,11.1,11.2,10.8,10.7,9.2,8.9,8.3,8.8,9.9,117.3
Average snowy days (≥ 0.1 in),3.5,2.8,1.1,0.1,0,0,0,0,0,0,0.3,1.7,9.5
Average relative humidity (%),63.2,61.3,59.2,58.9,66.1,68.4,69.1,71.1,71.3,69.5,66.5,65.5,65.8
Mean monthly sunshine hours,155.4,164.0,215.0,230.7,254.5,277.3,290.1,264.4,221.8,205.5,158.5,144.5,"2,581.7"
Percent possible sunshine,51,54,58,58,57,62,64,62,59,59,52,49,58


In [48]:
climate_df.rename(columns = name_change

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °F (°C),42.4(5.8),45.7(7.6),53.9(12.2),65.5(18.6),75.2(24.0),85.0(29.4),89.0(31.7),87.0(30.6),80.3(26.8),68.4(20.2),57.7(14.3),46.1(7.8),66.4(19.1)
Average low °F (°C),29.2(−1.6),31.4(−0.3),38.8(3.8),47.6(8.7),56.9(13.8),67.1(19.5),72.5(22.5),70.9(21.6),63.8(17.7),51.8(11.0),42.9(6.1),32.8(0.4),50.5(10.3)
Average precipitation inches (mm),2.92(74),2.60(66),3.86(98),3.22(82),3.49(89),3.27(83),4.62(117),3.39(86),4.09(104),3.05(77),2.97(75),3.41(87),"40.89(1,038)"
Average snowfall inches (cm),6.8(17),8.0(20),1.9(4.8),trace,0(0),0(0),0(0),0(0),0(0),0(0),0.4(1.0),3.0(7.6),20.1(51)
Average precipitation days (≥ 0.01 in),9.5,8.4,10.5,11.1,11.2,10.8,10.7,9.2,8.9,8.3,8.8,9.9,117.3
Average snowy days (≥ 0.1 in),3.5,2.8,1.1,0.1,0,0,0,0,0,0,0.3,1.7,9.5
Average relative humidity (%),63.2,61.3,59.2,58.9,66.1,68.4,69.1,71.1,71.3,69.5,66.5,65.5,65.8
Mean monthly sunshine hours,155.4,164.0,215.0,230.7,254.5,277.3,290.1,264.4,221.8,205.5,158.5,144.5,"2,581.7"
Percent possible sunshine,51,54,58,58,57,62,64,62,59,59,52,49,58


In [45]:
name_change

{'Average high °C (°F)': 'Average high °F (°C)',
 'Average low °C (°F)': 'Average low °F (°C)',
 'Average precipitation mm': 'Average precipitation inches (mm)',
 'Average precipitation days': 'Average precipitation days (≥ 0.01 in)'}

In [35]:
climate_df.rename(columns = name_change)

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °F (°C),42.4(5.8),45.7(7.6),53.9(12.2),65.5(18.6),75.2(24.0),85.0(29.4),89.0(31.7),87.0(30.6),80.3(26.8),68.4(20.2),57.7(14.3),46.1(7.8),66.4(19.1)
Average low °F (°C),29.2(−1.6),31.4(−0.3),38.8(3.8),47.6(8.7),56.9(13.8),67.1(19.5),72.5(22.5),70.9(21.6),63.8(17.7),51.8(11.0),42.9(6.1),32.8(0.4),50.5(10.3)
Average precipitation inches (mm),2.92(74),2.60(66),3.86(98),3.22(82),3.49(89),3.27(83),4.62(117),3.39(86),4.09(104),3.05(77),2.97(75),3.41(87),"40.89(1,038)"
Average snowfall inches (cm),6.8(17),8.0(20),1.9(4.8),trace,0(0),0(0),0(0),0(0),0(0),0(0),0.4(1.0),3.0(7.6),20.1(51)
Average precipitation days (≥ 0.01 in),9.5,8.4,10.5,11.1,11.2,10.8,10.7,9.2,8.9,8.3,8.8,9.9,117.3
Average snowy days (≥ 0.1 in),3.5,2.8,1.1,0.1,0,0,0,0,0,0,0.3,1.7,9.5
Average relative humidity (%),63.2,61.3,59.2,58.9,66.1,68.4,69.1,71.1,71.3,69.5,66.5,65.5,65.8
Mean monthly sunshine hours,155.4,164.0,215.0,230.7,254.5,277.3,290.1,264.4,221.8,205.5,158.5,144.5,"2,581.7"
Percent possible sunshine,51,54,58,58,57,62,64,62,59,59,52,49,58


In [33]:
climate_df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °F (°C),42.4(5.8),45.7(7.6),53.9(12.2),65.5(18.6),75.2(24.0),85.0(29.4),89.0(31.7),87.0(30.6),80.3(26.8),68.4(20.2),57.7(14.3),46.1(7.8),66.4(19.1)
Average low °F (°C),29.2(−1.6),31.4(−0.3),38.8(3.8),47.6(8.7),56.9(13.8),67.1(19.5),72.5(22.5),70.9(21.6),63.8(17.7),51.8(11.0),42.9(6.1),32.8(0.4),50.5(10.3)
Average precipitation inches (mm),2.92(74),2.60(66),3.86(98),3.22(82),3.49(89),3.27(83),4.62(117),3.39(86),4.09(104),3.05(77),2.97(75),3.41(87),"40.89(1,038)"
Average snowfall inches (cm),6.8(17),8.0(20),1.9(4.8),trace,0(0),0(0),0(0),0(0),0(0),0(0),0.4(1.0),3.0(7.6),20.1(51)
Average precipitation days (≥ 0.01 in),9.5,8.4,10.5,11.1,11.2,10.8,10.7,9.2,8.9,8.3,8.8,9.9,117.3
Average snowy days (≥ 0.1 in),3.5,2.8,1.1,0.1,0,0,0,0,0,0,0.3,1.7,9.5
Average relative humidity (%),63.2,61.3,59.2,58.9,66.1,68.4,69.1,71.1,71.3,69.5,66.5,65.5,65.8
Mean monthly sunshine hours,155.4,164.0,215.0,230.7,254.5,277.3,290.1,264.4,221.8,205.5,158.5,144.5,"2,581.7"
Percent possible sunshine,51,54,58,58,57,62,64,62,59,59,52,49,58
